# 协程（一）
版本：R0  
日期：2018-5-5

## 1 `yield from` vs `yield`

#### 委派生成器：
包含 `yield from <iterable>` 表达式的生成器函数。

#### 子生成器：
 `yield from <iterable>` 中的 `iterable` 获取的生成器。
 
 

In [2]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i

list(gen())

['A', 'B', 1, 2]

In [6]:
g = gen()
for i in range(4):
    print(next(g))

A
B
1
2


In [8]:
def gen():
    yield from 'AB'
    yield from range(1,3)

list(gen())

['A', 'B', 1, 2]

In [9]:
g = gen()
for i in range(4):
    print(next(g))

A
B
1
2


In [1]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 子生成器
def averager():
    total = 0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

# 委派生成器
def grouper(results, key):
    while True:
        results[key] = yield from averager()
        
# 客户端
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
        
    print(results)
    report(results)
    

def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print(f"{result.count} {group} averageing {result.average} {unit}")
        

data = {
    'girls;kg': [40, 38, 45, 50, 55, 60, 55, 43, 46, 51],
    'boys;kg': [50, 58, 45, 50, 55, 60, 55, 43, 46, 51],
    'girls;m': [1.58, 1.6, 1.62, 1.64, 1.55, 1.6, 1.59, 1.6, 1.64, 1.68],
    'boys;m': [1.68, 1.7, 1.62, 1.74, 1.75, 1.6, 1.79, 1.8, 1.64, 1.68],   
}

main(data)

{'girls;kg': Result(count=10, average=48.3), 'boys;kg': Result(count=10, average=51.3), 'girls;m': Result(count=10, average=1.61), 'boys;m': Result(count=10, average=1.7)}
10 boys averageing 51.3 kg
10 boys averageing 1.7 m
10 girls averageing 48.3 kg
10 girls averageing 1.61 m


## 2 `concurrent.futures.Future` vs `asyncio.Future`

### 2.0 处理流程
#### 2.0.1 `asyncio`
1. 在一个单线程程序中使用主循环依次激活队列里的协程。
2. 各个协程分别向前执行几步，然后把控制权让给主循环，主循环再激活队列里的下一个协程。

#### 2.0.2 `concurrent`

### 2.1 创建任务
#### `asyncio`
##### 1. `asyncio.async(coro_or_future, *, loop=None)`  
1.1  参数说明：  
    - `coro_or_future`: future 或 task 对象，就原封不动的返回；coro，则async函数会调用 loop.create_task(...) 方法创建 Task 对象。
    - `loop`: 参数可选，用于传入事件循环；如果没有传入，则 async 函数会通过调用 asyncio.get_event_loop() 函数获取循环对象。
    
##### 2. `BaseEventLoop.create_task(coro)`  
排定协程执行时间。  
2.1 参数说明：  
    - `coro`:    
2.2 返回值：
    - `asyncio.Task`: 对象。
  
### 2.2 处理结果